In [1]:
# pandasのインポート
import pandas as pd

# numpyのインポート
import numpy as np

# matplotlibのインポート
from matplotlib import pyplot as plt

In [2]:
train = pd.read_csv("/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/train.csv")
train_add = pd.read_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/train_add.csv')

test=pd.read_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/test.csv')

condition=pd.read_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/condition.csv')
condition_add=pd.read_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/condition_add.csv')

stadium=pd.read_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/stadium.csv')
data2014=pd.read_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/2014_add.csv')

sample=pd.read_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/sample_submit.csv', header=None)

In [3]:
train = pd.concat([train, train_add])
condition = pd.concat([condition, condition_add])

In [4]:
train = pd.merge(train, condition, on='id', how='inner')
test = pd.merge(test, condition, on='id', how='inner')

In [5]:
train = pd.merge(train, stadium, left_on='stadium', right_on='name', how='left')
test = pd.merge(test, stadium, left_on='stadium', right_on='name', how='left')

In [6]:
y=train['y']
train.drop(['y','id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)
all = pd.concat((train, test)).reset_index(drop=True)

In [7]:
# 'gameday'カラムから月を抽出し、整数型に変換して新たなカラム'month'を作成
all['month'] = all['gameday'].apply(lambda x: int(x[:2])).astype(int)

# 'gameday'カラムから日を抽出し、整数型に変換して新たなカラム'day'を作成
all['day'] = all['gameday'].apply(lambda x: int(x[3:5]))

# 'gameday'カラムから曜日を抽出して新たなカラム'week'を作成
all['week'] =all['gameday'].apply(lambda x: x[6:7])
all['match_num'] =all['match'].apply(lambda x: int(x[x.find("第")+1: x.find("節")])).astype(int)

In [8]:
def simplify_weather(weather):
    if '雪' in weather:
        return '雨'
    elif '雨' in weather:
        return '雨'
    elif '曇' in weather:
        return '曇'
    elif '晴' in weather:
        return '晴'
    else:
        return weather

all['weather'] = all['weather'].apply(simplify_weather)

In [9]:
all['humidity'] =all['humidity'].apply(lambda x: float(int(x[:-1])*0.01)).astype(int)
all['time'] =all['time'].apply(lambda x: x.split(':')[0]).astype(int).astype(int)
all['tv_num'] =all['tv'].apply(lambda x: len(x.split('／')))

all= all.drop(columns=['gameday','match','tv','name'])

In [10]:
all.drop(index=1385, inplace=True) #y=0の外れ値

In [11]:
all = all.replace('ザスパ草津','ザスパクサツ群馬')

In [12]:
all['J1_Nov'] = ((all['stage'] == 'Ｊ１') & (all['month'] == 11)).astype(int)
all['J1_Dec'] = ((all['stage'] == 'Ｊ１') & (all['month'] == 12)).astype(int)
all['J2_Nov'] = ((all['stage'] == 'Ｊ２') & (all['month'] == 11)).astype(int)
all = all.drop(columns='month')

In [13]:
use_columns = ['weather','home','away','stage','year','capa', 'match_num','J1_Nov','J1_Dec' ,'J2_Nov', 'time', 'tv_num','temperature','humidity','week']
all=all[use_columns]
all = pd.get_dummies(all, drop_first=True)

In [14]:
train =  all.head(1952)
test = all.iloc[1952:, :]
train['y']=y

<ipython-input-14-898d0dd7bc57>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['y']=y


In [15]:
X_train=train.drop(columns=['y'])
y_train=train['y']
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
predict_train=lr.predict(X_train)
predict_test=lr.predict(test)

In [16]:
y_test_series = pd.Series(predict_test)
result_df = pd.concat([sample.iloc[:, 0],y_test_series], axis=1)
result_df.to_csv('/content/drive/MyDrive/signate/練習 Jリーグの観客動員数予測/submit.csv', header=False, index=False)

In [17]:
tmp = pd.concat([X_train, y_train],axis=1)
tmp['pred'] = predict_train
tmp['res'] = y_train - predict_train

In [18]:
tmp = tmp.sort_values(by = 'res', ascending = False )
tmp.head(10)

,year,capa,match_num,J1_Nov,J1_Dec,J2_Nov,time,tv_num,temperature,humidity,...,stage_Ｊ２,week_日,week_月,week_木,week_水,week_火,week_金,y,pred,res
1844,2013,72327,33,1,0,0,14,3,12.7,0,...,0,0,0,0,0,0,0,62632,29647.860026,32984.139974
1443,2014,40000,11,0,0,0,19,4,17.8,0,...,0,0,0,0,0,0,0,38966,23058.923695,15907.076305
1409,2014,47816,7,0,0,0,14,3,18.0,0,...,0,0,0,0,0,0,0,42723,26972.790934,15750.209066
693,2013,63700,2,0,0,0,16,3,21.6,0,...,0,0,0,0,0,0,0,52293,37575.485149,14717.514851
1418,2014,49970,8,0,0,0,16,2,13.2,0,...,0,0,0,0,0,0,0,40761,26766.935323,13994.064677
1359,2014,47816,1,0,0,0,14,3,12.3,0,...,0,0,0,0,0,0,0,37079,23363.242010,13715.757990
586,2012,40000,32,0,0,0,18,3,26.4,0,...,1,1,0,0,0,0,0,23617,10174.526536,13442.473464
1384,2014,40728,4,0,0,0,15,2,13.9,0,...,0,1,0,0,0,0,0,32099,18755.606493,13343.393507
1463,2014,63700,14,0,0,0,14,3,23.6,0,...,0,0,0,0,0,0,0,54350,41487.931630,12862.068370
895,2013,63700,28,0,0,0,16,3,19.9,0,...,0,0,0,0,0,0,0,47790,35319.824854,12470.175146


In [19]:
sns.boxplot(data=tmp[tmp['stage_Ｊ２'] == 1], x='month', y='y')
# 可視化結果を表示する為に必要な関数
plt.show()
sns.boxplot(data=tmp[tmp['stage_Ｊ２'] == 0], x='month', y='y')
# 可視化結果を表示する為に必要な関数
plt.show()


NameError: name 'sns' is not defined